# Prototype - *get_patch_location_array()* function
    * Common to all 4 patch extraction use cases.
    * Decouples selection image mask & (mask size) from individual use case flows.
    * Insertion point for new or modified location-selection methods.
    * Allow inspection / select-method parameter modification before making output.
    * Allows paralellization of patch location (think slower more specific select methods). <br>
****
## Assemble and create unit test data:
### Redo unit test for **get_sample_selection_mask()** (output type change)
****
### First Edit:
```python
def get_patch_location_array(run_parameters):
    """ Usage: patch_location_array = get_patch_location_array(run_parameters)
    Args:
        run_parameters:     keys:
                            image_file_name,
                            thumbnail_divisor,
                            patch_select_method,
                            patch_height,
                            patch_width
    Returns:
        patch_location_array
        
    """
    patch_location_array = []
    
    image_file_name = run_parameters['image_file_name']
    thumbnail_divisor = run_parameters['thumbnail_divisor']
    patch_select_method = run_parameters['patch_select_method']
    patch_height = run_parameters['patch_height']
    patch_width = run_parameters['patch_width']
    
    #                     OpenSlide open                      #
    os_im_obj = openslide.OpenSlide(image_file_name)
    
    pixels_height = os_im_obj.dimensions[1]
    rows_fence_array = get_fence_array(patch_length=patch_height, overall_length=pixels_height)
    
    pixels_width = os_im_obj.dimensions[0]
    cols_fence_array = get_fence_array(patch_length=patch_width, overall_length=pixels_width)
        
    small_im = os_im_obj.get_thumbnail((pixels_height // thumbnail_divisor, 
                                        pixels_width // thumbnail_divisor))
    os_im_obj.close()
    #                     OpenSlide close                     #
    
    mask_im = get_sample_selection_mask(small_im, patch_select_method)
        
    it_rows = zip(rows_fence_array[:,0] // thumbnail_divisor, 
                  rows_fence_array[:,1] // thumbnail_divisor, 
                  rows_fence_array[:,0])
    
    lft_cols = cols_fence_array[:,0] // thumbnail_divisor
    rgt_cols = cols_fence_array[:,1] // thumbnail_divisor
    cols_array = cols_fence_array[:,0]
    
    for tmb_row_top, tmb_row_bot, row_n in it_rows:
        it_cols = zip(lft_cols, rgt_cols, cols_array)
        for tmb_col_lft, tmb_col_rgt, col_n in it_cols:
            if (mask_im[tmb_row_top:tmb_row_bot, tmb_col_lft:tmb_col_rgt]).sum() > 0:
                patch_location_array.append((row_n, col_n))
                
    return patch_location_array
```

In [ ]:
import time
import os
import sys
import openslide

sys.path.insert(0, '../src/python')
from digipath_toolkit import *

data_dir = '../../DigiPath_MLTK_data/Aperio'
file_type_list=['.svs', '.tif', '.tiff']
fs_od = get_file_size_ordered_dict(data_dir, file_type_list)
list_number = 0
for k, v in fs_od.items():
    print('%3i %30s: %i'%(list_number, k,v))
    list_number += 1

In [ ]:
#                         test data file for threshold_otsu
run_parameters = {}

data_dir = '../data/images/' 
image_file_name = 'CMU-1-Small-Region.svs'
run_parameters['wsi_filename'] = os.path.join(data_dir, image_file_name)
print('Image File:\n', run_parameters['wsi_filename'])

run_parameters['thumbnail_divisor'] = 10
run_parameters['patch_select_method'] = 'threshold_otsu' # 'threshold_rgb2lab'
run_parameters['patch_height'] = 224
run_parameters['patch_width'] = 224

"""
                output & time test:                CALL - - get_patch_location_array()
"""
t0 = time.time()
patch_location_array = get_patch_location_array(run_parameters)
tt = time.time() - t0

print('%i images found\t%0.3f s\n'%(len(patch_location_array), tt))

print('type(patch_location_array) =',type(patch_location_array), '\t', len(patch_location_array),'\n')
for d in patch_location_array:
    print(type(d), '\t', d, '\t', '\t', d[0], d[0])

In [ ]:
pl_arr = np.array(patch_location_array)
pl_arr.shape

In [ ]:
pl_BAK = list(pl_arr)
for d in pl_BAK:
    print(tuple(d))

In [ ]:
for r, c in patch_location_array[:]:
    print(r,c)

In [ ]:
patch_location_array[:]